In [1]:
### Q1
## Regression multilinéaire

### var dépendantes
### - 'Débit sève (L/j)'
### - 'Sucre sève (%)'
### - 'Transmittance produit (%)'
### - 'Production moyenne par entaille (L)'

In [78]:
### Importer les bibliothèques

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.api as stm
import statsmodels.stats.weightstats as ws
import sklearn.linear_model as sk_lin_mod
import math
import seaborn as sns
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import OLSInfluence
#from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression

import PyQt5 #works
from PyQt5 import QtCore, QtGui, QtWidgets #works for pyqt5
%matplotlib 

Using matplotlib backend: QtAgg


In [79]:
### chercher le csv
print (os.getcwd())
currDir = os.getcwd()
fullPath = currDir + "/CabaneASucrev0r3.csv"
#type(currDir)

donnee = pd.read_csv(fullPath)
stats=donnee.describe().apply(lambda x: round(x, 2))
stats


/home/hadoop/Git_Ia/CegepSteFoy_IA/04_Eda/TP02


,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pixel6_B,Pixel7_R,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B
count,1875.00,1875.00,1873.00,1875.00,1873.00,1875.00,1872.00,1875.00,1875.00,1875.00,...,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00
mean,2010.00,45.14,1.52,-8.63,-3.56,10.15,2.74,610.33,20.63,26.52,...,33.77,157.94,71.32,32.71,156.87,70.61,33.46,157.79,70.61,32.82
std,6.06,25.78,7.64,8.70,7.84,4.77,5.57,126.97,5.25,10.23,...,26.42,85.97,52.30,26.69,85.23,52.97,27.30,85.76,53.76,27.04
min,2000.00,1.00,-22.60,-34.80,-25.70,0.90,0.00,313.20,14.00,11.57,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2005.00,23.00,-3.60,-15.10,-9.30,6.70,0.00,536.40,16.00,17.28,...,8.35,111.90,32.56,4.86,111.96,31.29,5.54,109.94,28.83,4.33
50%,2010.00,45.00,1.70,-6.70,-2.50,9.60,0.00,620.30,20.00,26.46,...,32.97,183.19,71.92,31.11,180.81,69.96,31.41,183.02,70.56,31.11
75%,2015.00,67.00,6.40,-1.45,2.40,13.10,2.60,685.10,25.00,34.32,...,55.82,227.42,102.46,54.91,224.96,103.02,56.40,225.82,104.85,55.03
max,2020.00,90.00,27.50,10.00,18.80,31.00,41.60,811.70,30.00,46.78,...,108.80,255.00,255.00,125.40,255.00,255.00,117.94,255.00,255.00,132.12


In [80]:
### supprimer les colonnes pixels
donnee.filter(like='Pixel').columns
dfSansPixel = donnee.drop(donnee.filter(like='Pixel').columns, axis=1)
dfSansPixel.columns

Index(['Date', 'Année', 'Jour Calendrier Saison', 'Temp max.(°C)',
       'Temp min.(°C)', 'Temp moy.(°C)', 'Diff Temp (°C)', 'Précip. tot. (mm)',
       'Précip. Tot. Hiver (mm)', 'Nombre épisodes gel/dégel',
       'Production moyenne par entaille (L)', 'Débit sève (L/j)',
       'Sucre sève (%)', 'Alimentation osmoseur (L/j)',
       'Osmoseur (heures opération/j)', 'Pression osmoseur (bar)',
       'Sucre sortie osmoseur (%)', 'Température Bouilloire (0C)',
       'Temps bouilloire (h)', 'Sucre du sirop obtenu (%)',
       'Quantité de sirop obtenue (L)', 'Transmittance produit (%)',
       'Classe Sirop'],
      dtype='object')

In [81]:
### liste de variables (colonnes) dépendantes
list_cols_dependennt_vars = []
col_debit_seve = dfSansPixel['Débit sève (L/j)'].name
col_sucre_dans_seve = dfSansPixel['Sucre sève (%)'].name
col_pct_transmittance = dfSansPixel['Transmittance produit (%)'].name
col_productivite_seve_par_saison = dfSansPixel['Production moyenne par entaille (L)'].name
list_cols_dependennt_vars = [col_debit_seve, col_sucre_dans_seve, col_pct_transmittance, col_productivite_seve_par_saison]

In [82]:
## convertir Classe Sirop en numérique - https://www.youtube.com/watch?v=wH_ezgftiy0&t=136s
dfSansPixel['CategClasseSirop'] = dfSansPixel['Classe Sirop'].astype('category').cat.codes
dfNumerique = dfSansPixel.select_dtypes(exclude='object').copy()
dfNumerique.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%),CategClasseSirop
count,1875.00,1875.00,1873.00,1875.00,1873.00,1875.00,1872.00,1875.00,1875.00,1875.00,...,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00
mean,2010.00,45.14,1.52,-8.63,-3.56,10.15,2.74,610.33,20.63,26.52,...,1038.45,2.34,34.06,6.84,88.52,2.02,57.02,74.69,37.08,2.12
std,6.06,25.78,7.64,8.70,7.84,4.77,5.57,126.97,5.25,10.23,...,1745.36,3.93,14.39,3.42,37.10,1.72,23.91,126.71,18.92,1.25
min,2000.00,1.00,-22.60,-34.80,-25.70,0.90,0.00,313.20,14.00,11.57,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2005.00,23.00,-3.60,-15.10,-9.30,6.70,0.00,536.40,16.00,17.28,...,121.00,0.27,37.72,5.72,103.82,1.53,66.00,7.86,29.81,1.00
50%,2010.00,45.00,1.70,-6.70,-2.50,9.60,0.00,620.30,20.00,26.46,...,276.00,0.62,39.60,7.60,104.03,1.62,67.00,19.20,40.70,3.00
75%,2015.00,67.00,6.40,-1.45,2.40,13.10,2.60,685.10,25.00,34.32,...,669.00,1.50,41.12,9.12,104.21,1.69,68.00,44.90,49.60,3.00
max,2020.00,90.00,27.50,10.00,18.80,31.00,41.60,811.70,30.00,46.78,...,10664.62,24.00,46.59,14.59,104.69,11.48,70.00,681.99,82.52,4.00


In [83]:
dfNumerique.isnull().sum()

Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          2
Temp min.(°C)                          0
Temp moy.(°C)                          2
Diff Temp (°C)                         0
Précip. tot. (mm)                      3
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
CategClasseSirop                       0
dtype: int64

In [84]:
## Comme il n'y a pas beaucoup d'enregistrements ayant null, on les ignore
dfNumerique.dropna(inplace=True)
dfNumerique.isnull().sum()

Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
CategClasseSirop                       0
dtype: int64

In [85]:
### "Considérerz retires les données où il n'y a pas de production (débit nul de sève et de sirop)"
dfNumerique = dfNumerique[dfNumerique['Débit sève (L/j)'] >0]
len(dfNumerique)
                

1592

In [86]:
##### séparation variables dépendantes (y) et indépendates (x)

## seulement les variables X
dfNumerique_X_cols = dfNumerique.loc[:, ~dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

lstColTemp = ['Temp max.(°C)', 'Temp min.(°C)']
dfNumerique_X_cols = dfNumerique_X_cols.loc[:,~dfNumerique_X_cols.columns.isin(lstColTemp) ].copy()

## seulement les variables Y
dfNumerique_Y_cols = dfNumerique.loc[:, dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

len(dfNumerique.columns)

22

In [87]:
### Fonction qui imprime le sommaire du modèle
def print_model_summary(pDfColsX, pDfColsY):
    ## Add constant
    pDfColsX = stm.add_constant(pDfColsX)
    lst_models = []
    for y in (pDfColsY.columns.values):
    
        col_y = dfNumerique_Y_cols[y]
    
        model = stm.OLS(col_y, pDfColsX.assign(const=1)).fit()
        print("=========")
        print("var depend: ", y)
    
        #lst_models.append(model)
        print(model.summary())
        #sns.distplot(model.resid, fit=sts.norm)        
        #p = sns.histplot(model.resid, kde=True, stat="density").set(title = y)

In [88]:
print("---- Sommaire dfNumerique ----")
print_model_summary(dfNumerique_X_cols, dfNumerique_Y_cols)

---- Sommaire dfNumerique ----
var depend:  Production moyenne par entaille (L)
                                     OLS Regression Results                                    
Dep. Variable:     Production moyenne par entaille (L)   R-squared:                       0.948
Model:                                             OLS   Adj. R-squared:                  0.947
Method:                                  Least Squares   F-statistic:                     1898.
Date:                                 Sat, 03 Dec 2022   Prob (F-statistic):               0.00
Time:                                         13:09:32   Log-Likelihood:                -3588.5
No. Observations:                                 1592   AIC:                             7209.
Df Residuals:                                     1576   BIC:                             7295.
Df Model:                                           15                                         
Covariance Type:                             nonrobust  

In [89]:
###
### À ce moment, on a un modèle pour chacune de variable dépendantes
### avec une très bonne précision (Adj. R-squared) 
### Le plus bas de Adj. R squared est 0.707 pour la variable dépendante Transmittance produit (%)

### Dans tous les cas, on a potentiellement un problème d'haute colinéarité 
### "This might indicate that there are
### strong multicollinearity problems or that the design matrix is singular."

#### On essaie d'améliorer le modèle

In [90]:
### vérification de la colinéarité
### on peut voir dans le heatmap que, effectivement, on
### des cas de haute colinéarité, parfois même parfaite
corr = dfNumerique.corr()
sns.heatmap(corr, cbar=True, cmap="Blues", center=0, annot=True, fmt=".1f")
#print(corr)

<AxesSubplot: >

In [91]:
#### Obtenons un dataframe plus compacte et sans le problème de colinéarité
## La méthode utilisé est inspirée de: 
### https://github.com/simaria22/prediction_heart_failure/blob/master/feature_select.py

## Ajoute des colonnes au modèle pendant que la nouvelle colonne améliore
## le modèle OU
## jusqu'à qu'un certain niveau de précision soit atteint. Ici on a arbitre que 70% serait assez bon
def get_reduced_df_X(pDfx, pY, pAccuracy=0.7):
    print("------")
    print("dependent var y: ", pY.name)
    
    
    
    # new X df with fewer columns than pDfx
    reducedDf = []

    # column names to keep
    dfKeeper = None

    #no of features
    nof_list=np.arange(1,len(pDfx.columns))            
    high_score=0
    #Variable to store the optimum features
    nof=0           
    score_list =[]
    for n in range(len(nof_list)):
        X_train, X_test, y_train, y_test = train_test_split(pDfx, pY, test_size = 0.3, random_state = 0)
        model = LinearRegression()
        rfe = RFE(model,n_features_to_select= nof_list[n])
        X_train_rfe = rfe.fit_transform(X_train,y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe,y_train)
        score = model.score(X_test_rfe,y_test)
        score_list.append(score)
        
        print("n: ", n)
        print("score: ", score)
        print("high_score: ", high_score)

        # print summaries for the selection of attributes
        print("rfe.support_: ",rfe.support_)
        print("rfe.ranking_: " ,rfe.ranking_)
        
        #dfKeeper = pd.DataFrame(rfe.support_,index=pDfx.columns,columns=['Rank'])
        data= {'colName': pDfx.columns, 'keep': rfe.support_ }
        dfKeeper = pd.DataFrame(data)
        dfKeeper = dfKeeper[dfKeeper["keep"]==True]
        if(score>high_score):
            high_score = score
            nof = nof_list[n]
        if (score > pAccuracy):
            break

    
    print("Optimum number of features: %d" %nof)
    print("Score with %d features: %f" % (nof, high_score))
    print("df columns to keep: ")
    print("shape")
    print(dfNumerique_X_cols[dfKeeper["colName"]].shape)
    print(dfKeeper)
    
    reducedDf = dfNumerique_X_cols[dfKeeper["colName"]]
    
    #print("reducedDf: ", reducedDf.head())
    #print("unique: ", reducedDf[dfKeeper["colName"]].unique())
    return reducedDf


In [93]:
##### Ensuite, pour chaque variable dépendante, on obtient son ensemble de variables
##### indépendantes

### création d'un dictionnaire contenant la variable dépendantes et ses variables indépendantes
### correspondantes
### clé = colonne dépendante, valeur = dataframe des colonnes avec haute influence

dict_y_colsX = {}

for y in (dfNumerique_Y_cols.columns.values):    
    col_y = dfNumerique_Y_cols[y]
    reducedDf = get_reduced_df_X(dfNumerique_X_cols, col_y)
    #print("reduceDf.columns.values: ",  reducedDf.columns.values)
    #print("unique values: ", reducedDf[reducedDf.columns.values[0]].unique())
    #print("len: ", len(reducedDf[reducedDf.columns.values[0]]))
    #print("Nan: ", reducedDf[reducedDf.columns.values[0]].isnull().sum())
    ## call to analyze_model
    #analyze_model(dfNumerique_Y_cols, reducedDf)
    #analyze_model(col_y.to_frame(), reducedDf)
    dict_y_colsX.update({col_y.name: reducedDf})

------
dependent var y:  Production moyenne par entaille (L)
n:  0
score:  0.9169477115680265
high_score:  0
rfe.support_:  [False False False False False False  True False False False False False
 False False False False]
rfe.ranking_:  [ 2 15 13  6 16 10  1  8  7 11  9  3  5  4 12 14]
Optimum number of features: 1
Score with 1 features: 0.916948
df columns to keep: 
shape
(1592, 1)
                     colName  keep
6  Nombre épisodes gel/dégel  True
------
dependent var y:  Débit sève (L/j)
n:  0
score:  0.9487611224637206
high_score:  0
rfe.support_:  [False False False False False False False False False False False False
  True False False False]
rfe.ranking_:  [12 15 13  7 10 16  8 14  2  6 11  4  1  5  9  3]
Optimum number of features: 1
Score with 1 features: 0.948761
df columns to keep: 
shape
(1592, 1)
                 colName  keep
12  Temps bouilloire (h)  True
------
dependent var y:  Sucre sève (%)
n:  0
score:  0.4469431677133521
high_score:  0
rfe.support_:  [False Fal

In [94]:
###### Une fois ayant eliminé la colinéarité, on vérifie si on obtient des meilleurs résultats
######
######
for key in (dict_y_colsX.keys()):
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    print_model_summary(dfX, dfY.to_frame())

key:  Production moyenne par entaille (L)
var depend:  Production moyenne par entaille (L)
                                     OLS Regression Results                                    
Dep. Variable:     Production moyenne par entaille (L)   R-squared:                       0.910
Model:                                             OLS   Adj. R-squared:                  0.910
Method:                                  Least Squares   F-statistic:                 1.606e+04
Date:                                 Sat, 03 Dec 2022   Prob (F-statistic):               0.00
Time:                                         13:09:38   Log-Likelihood:                -4019.0
No. Observations:                                 1592   AIC:                             8042.
Df Residuals:                                     1590   BIC:                             8053.
Df Model:                                            1                                         
Covariance Type:                             

In [95]:
#####
##### À ce point, on encore un modèle avec une bonne précision (Le "Adj. R-squared" moins bon
#### est 0.682)
####
#### On a beaucoup moins de variables indépendantes
#### Seuelement la variable 'Transmittance produit (%)' contient un advertissement de colinéarité
#####

In [ ]:
### Réglons le problème de multicolinéarité
### pour la var dépendante transmittance

In [98]:
### vif
## https://www.kdnuggets.com/2019/07/check-quality-regression-model-python.html
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [99]:
### https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = dfNumerique_X_cols.columns


In [100]:
# calculating VIF for each feature

vif_data["VIF"] = [vif(dfNumerique_X_cols.values, i)
                          for i in range(len(dfNumerique_X_cols.columns))]

vif_data.sort_values(by="VIF", axis=0, kind="quicksort",  ascending=False)

,feature,VIF
9,Pression osmoseur (bar),1.627238e+07
7,Alimentation osmoseur (L/j),2.045958e+06
8,Osmoseur (heures opération/j),2.045640e+06
10,Sucre sortie osmoseur (%),6.953414e+05
14,Quantité de sirop obtenue (L),5.208656e+01
11,Température Bouilloire (0C),5.070014e+01
13,Sucre du sirop obtenu (%),5.063313e+01
12,Temps bouilloire (h),2.311587e+01
5,Précip. Tot. Hiver (mm),2.851839e+00
0,Année,2.796179e+00


In [105]:
dfHighVif = vif_data[vif_data["VIF"] >10]
len(dfHighVif)

8

In [106]:
### dfNumerique_X sans les vif élevés
dfNumerique_X_cols_vif =[]
dfNumerique_X_cols_vif = dfNumerique_X_cols.loc[:, ~dfNumerique_X_cols.columns.isin(dfHighVif["feature"].values)].copy()
#for x in (dfVifPlusGrand10["feature"].values):
#    print("col: ", x)
dfNumerique_X_cols_vif.columns

Index(['Année', 'Jour Calendrier Saison', 'Temp moy.(°C)', 'Diff Temp (°C)',
       'Précip. tot. (mm)', 'Précip. Tot. Hiver (mm)',
       'Nombre épisodes gel/dégel', 'CategClasseSirop'],
      dtype='object')

\-------------------\

Est-ce que l'on obtien un résultat meilleur pour 'Transmittance produit (%)' ?

\----------------\

In [107]:
### appelle get_reduced_df_X avec 
### X =  dfNumerique_X_cols_vif et Y = dfNumerique_Y_cols['Transmittance produit (%)']
newDfTransmittance = get_reduced_df_X(dfNumerique_X_cols_vif, dfNumerique_Y_cols['Transmittance produit (%)'])
print_model_summary(newDfTransmittance, dfNumerique_Y_cols['Transmittance produit (%)'].to_frame())

------
dependent var y:  Transmittance produit (%)
n:  0
score:  0.6731022804420081
high_score:  0
rfe.support_:  [False False False False False False False  True]
rfe.ranking_:  [5 4 2 3 6 8 7 1]
n:  1
score:  0.6735274216661938
high_score:  0.6731022804420081
rfe.support_:  [False False  True False False False False  True]
rfe.ranking_:  [4 3 1 2 5 7 6 1]
n:  2
score:  0.6730320241677423
high_score:  0.6735274216661938
rfe.support_:  [False False  True  True False False False  True]
rfe.ranking_:  [3 2 1 1 4 6 5 1]
n:  3
score:  0.6808052762717531
high_score:  0.6735274216661938
rfe.support_:  [False  True  True  True False False False  True]
rfe.ranking_:  [2 1 1 1 3 5 4 1]
n:  4
score:  0.6810675536688185
high_score:  0.6808052762717531
rfe.support_:  [ True  True  True  True False False False  True]
rfe.ranking_:  [1 1 1 1 2 4 3 1]
n:  5
score:  0.6808252604957085
high_score:  0.6810675536688185
rfe.support_:  [ True  True  True  True  True False False  True]
rfe.ranking_:  [1 1 1

In [ ]:
##### Ce modèle 
#####
#####

In [96]:
### mise à l'échelle des données
#def normalize_df(pDf):
#    scaler = StandardScaler()
#    scaler.fit(pDf)
#    scaled = scaler.fit_transform(pDf)
#    pDfNorm = pd.DataFrame(scaled, columns=pDf.columns)
   
#    return pDfNorm

In [24]:
#dfNumeriqueNorm = normalize_df(dfNumerique)

In [26]:
#dfNumeriqueNorm.describe().apply(lambda x: round(x,2))

In [ ]:
###############################################